In [1]:
#Embedding generation using emb_dataloader which checks the path and verifies the file type and images

In [15]:
# some packages need async support in jupyter
import nest_asyncio
nest_asyncio.apply()

In [2]:
# layer_name can also be given 
# for name, module in model.named_modules():
#     if isinstance(module, torch.nn.AdaptiveAvgPool2d):  # Look for avgpool layers
#         print(name, module)
# output:
# avgpool AdaptiveAvgPool2d(output_size=(1, 1))
# layer4.2.downsample.1 AdaptiveAvgPool2d(output_size=(1, 1))

## layer_name = 'self.model.layer4[2].downsample[1]' or layer='avgpool'

In [5]:
from mb_pytorch.models.modelloader import ModelLoader
from mb_pytorch.utils.generate_emb import EmbeddingGenerator
from mb_pytorch.utils.yaml_reader import YAMLReader
from mb_utils.src.logging import logger

In [6]:
yaml_read = YAMLReader('../scripts/loader_y.yaml') #load the data
yaml_data = yaml_read.read()

In [8]:
yaml_data['model']

{'model_type': 'detection',
 'model': 'faster_rcnn',
 'use_pretrained': True,
 'use_own_model': False,
 'model_path': '/home/malav/Desktop/mb_packages/mb_pytorch/models/...',
 'model_layer': 'avgpool',
 'device': 'cpu',
 'load_model': False,
 'use_torchvision_models': True,
 'model_name': 'fasterrcnn_resnet50_fpn',
 'model_version': '50',
 'model_backbone': 'resnet',
 'model_optimizer': 'Adam',
 'model_loss': 'cross_entropy',
 'model_scheduler': 'StepLR',
 'model_pretrained': True,
 'model_epochs': 3,
 'model_num_classes': 41,
 'model_initialization': 'None',
 'model_meta_data': {'model_bbox_threshold': 0.8},
 'model_train_parameters': {'Adam': {'lr': 0.001,
   'betas': [0.9, 0.999],
   'eps': 1e-08,
   'weight_decay': 0,
   'amsgrad': False},
  'SGD': {'lr': 0.01,
   'momentum': 0.9,
   'dampening': 0,
   'weight_decay': 0,
   'nesterov': False},
  'RMSprop': {'lr': 0.01,
   'alpha': 0.99,
   'eps': 1e-08,
   'weight_decay': 0,
   'momentum': 0,
   'centered': False},
  'cross_entropy

In [3]:
ModelLoader?

Init signature: ModelLoader(data: dict, logger=None)
Docstring:     
Base class for all neural network modules.

Your models should also subclass this class.

Modules can also contain other Modules, allowing to nest them in
a tree structure. You can assign the submodules as regular attributes::

    import torch.nn as nn
    import torch.nn.functional as F

    class Model(nn.Module):
        def __init__(self):
            super().__init__()
            self.conv1 = nn.Conv2d(1, 20, 5)
            self.conv2 = nn.Conv2d(20, 20, 5)

        def forward(self, x):
            x = F.relu(self.conv1(x))
            return F.relu(self.conv2(x))

Submodules assigned in this way will be registered, and will have their
parameters converted too when you call :meth:`to`, etc.

.. note::
    As per the example above, an ``__init__()`` call to the parent class
    must be made before assignment on the child.

:ivar training: Boolean represents whether this module is in training or
                

In [11]:
model_init = ModelLoader(yaml_data['model'])

In [13]:
model = model_init.get_model()

/home/malav/.local/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/malav/.local/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Wed 08:58:05,735 INF Model fasterrcnn_resnet50_fpn loaded from torchvision.models.                                                   


In [17]:
from mb_pytorch.dataloader.loader import BaseDataset, TorchDataLoader
train_dataset = BaseDataset(data_config=yaml_data['data']['file'], task_type=yaml_data['model']['model_type'], 
                            transform=yaml_data['transformation'],is_train=True,logger=logger)
train_dataloader= TorchDataLoader(dataset=train_dataset, batch_size=yaml_data['train_params']['batch_size'],
                                  shuffle=yaml_data['train_params']['shuffle'],num_workers=yaml_data['train_params']['num_workers'])

Wed 09:00:26,852 INF Loading data from /home/malav/Desktop/mb_packages/mb_pytorch/data/fruit_dataset/fruit_dataset_final.csv         
Wed 09:00:26,855 INF Loading DataFrame from /home/malav/Desktop/mb_packages/mb_pytorch/data/fruit_dataset/fruit_dataset_final.csv    
Wed 09:00:26,861 INF Successfully loaded DataFrame with shape (300, 5)                                                               
Wed 09:00:26,866 INF Checking duplicates for columns: ['image_path']                                                                 
Wed 09:00:26,868 INF No duplicates found                                                                                             
Wed 09:00:26,870 INF Removing unnamed columns                                                                                        


In [5]:
EmbeddingGenerator?

Init signature:
EmbeddingGenerator(
    model,
    dataloader,
    layer='avgpool',
    layer_name=None,
    device='cpu',
    logger=None,
) -> None
Docstring:      <no docstring>
Init docstring:
Initialize the EmbeddingGenerator class
Input:
    model: model for which embeddings are to be generated
    dataloader: dataloader for the data
    layer: layer from which embeddings are to be generated. Default: avgpool. if None, then it will look for the layer_name
    layer_name: name of the layer. Only used if layer is None. Default: None. Eg: 'self.model.layer4[2].downsample[1]'
    device: device on which the model is to be run
    logger: logger instance
File:           ~/.local/lib/python3.11/site-packages/mb_pytorch/utils/generate_emb.py
Type:           type
Subclasses:     

In [21]:
emb_init = EmbeddingGenerator(model,dataloader=train_dataloader,layer='avgpool',device='cpu')

In [26]:
model.modules.get_layer('avgpool')

AttributeError: 'function' object has no attribute 'get_layer'

In [22]:
emb_data = emb_init.generate_emb()

AttributeError: 'NoneType' object has no attribute 'register_forward_hook'

In [ ]:
emb_data